In [64]:
import os
import sklearn
import pandas as pd
import numpy as np
from __future__ import division
import collections
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import tree
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import sys
from sklearn.ensemble import GradientBoostingRegressor
import math
import csv
%matplotlib inline
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
import urllib
from sklearn.svm import LinearSVC


## Folder and path parameters

In [65]:
data_dir = "./Data"
kaggle_dir = os.path.join(data_dir, "Kaggle")
kenpom_dir = os.path.join(data_dir, "Kenpom")
sr_dir = os.path.join(data_dir, "Sports_ref")

In [66]:
num_team_stats = 14
teams_pd = pd.read_csv(os.path.join(kaggle_dir, "Teams.csv"))
teams_list = teams_pd["TeamName"].tolist()
print(teams_list[0:10])

['Abilene Chr', 'Air Force', 'Akron', 'Alabama', 'Alabama A&M', 'Alabama St', 'Albany NY', 'Alcorn St', 'Alliant Intl', 'American Univ']


## Using Pandas to visualize data

In [67]:
reg_season_compact_pd = pd.read_csv(kaggle_dir + "/RegularSeasonCompactResults.csv")
reg_season_compact_pd.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [68]:
reg_season_detailed_pd = pd.read_csv(kaggle_dir + '/RegularSeasonDetailedResults.csv')
reg_season_detailed_pd.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [69]:
seasons_pd = pd.read_csv(kaggle_dir + '/Seasons.csv')
seasons_pd.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


In [70]:
teams_pd = pd.read_csv(kaggle_dir + '/Teams.csv')
teamList = teams_pd['TeamName'].tolist()
teams_pd["TeamName"] = teams_pd["TeamName"].replace("Chr", "Chrawoiefjoa")
print(teams_pd.head())
print(teamList[0])

   TeamID     TeamName  FirstD1Season  LastD1Season
0    1101  Abilene Chr           2014          2018
1    1102    Air Force           1985          2018
2    1103        Akron           1985          2018
3    1104      Alabama           1985          2018
4    1105  Alabama A&M           2000          2018
Abilene Chr


In [71]:
sr_pd = pd.read_csv(sr_dir + "/2002_basic.csv", skiprows=1)
sr_pd.head()

,Rk,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Air Force,28,9,19,0.321,-0.95,3.23,3.0,11.0,...,331,465,0.712,154.0,614,327,228,39,390,NaN
1,2,Akron,31,10,21,0.323,-4.84,0.36,5.0,13.0,...,554,731,0.758,295.0,933,416,180,77,450,NaN
2,3,Alabama A&M,29,19,10,0.655,-11.33,-15.81,12.0,6.0,...,478,711,0.672,506.0,1217,426,395,116,504,NaN
3,4,Alabama-Birmingham,30,13,17,0.433,5.56,2.96,6.0,10.0,...,428,623,0.687,376.0,1117,433,231,87,448,NaN
4,5,Alabama State,32,19,13,0.594,-13.25,-14.15,12.0,6.0,...,483,755,0.640,389.0,1167,314,217,89,538,NaN


In [72]:
sr_pd = pd.read_csv(sr_dir + "/2002_adv.csv", skiprows=1)
sr_pd.head()

,Rk,School,G,W,L,W-L%,SRS,SOS,W.1,L.1,...,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,1,Air Force,28,9,19,0.321,-0.95,3.23,3.0,11.0,...,0.485,0.585,42.3,61.4,NaN,3.2,0.553,22.1,NaN,0.288
1,2,Akron,31,10,21,0.323,-4.84,0.36,5.0,13.0,...,0.311,0.559,47.5,58.7,NaN,4.4,0.506,18.9,NaN,0.350
2,3,Alabama A&M,29,19,10,0.655,-11.33,-15.81,12.0,6.0,...,0.339,0.494,49.3,52.3,NaN,6.7,0.458,17.7,NaN,0.238
3,4,Alabama-Birmingham,30,13,17,0.433,5.56,2.96,6.0,10.0,...,0.332,0.512,50.5,58.6,NaN,5.2,0.477,17.9,NaN,0.243
4,5,Alabama State,32,19,13,0.594,-13.25,-14.15,12.0,6.0,...,0.317,0.514,51.2,43.9,NaN,4.9,0.479,20.9,NaN,0.287


In [73]:
kenpom_pd = pd.read_csv(kenpom_dir + "/summaries/summary02.csv")
kenpom_pd.head()

,Season,TeamName,Tempo,RankTempo,AdjTempo,RankAdjTempo,OE,RankOE,AdjOE,RankAdjOE,DE,RankDE,AdjDE,RankAdjDE,AdjEM,RankAdjEM
0,2002,Air Force,55.7833,327,54.1733,327,99.4506,190,103.8170,149,106.8100,283,105.4580,214,-1.64046,181
1,2002,Akron,66.7009,260,65.6394,251,102.6500,128,105.2430,130,110.8370,313,111.1810,295,-5.93802,227
2,2002,Alabama,69.7639,171,68.0142,170,108.2290,42,114.0460,24,96.0457,56,95.3126,32,18.73300,22
3,2002,Alabama A&M,81.6407,1,78.9833,1,97.8208,220,92.1604,304,90.6649,4,101.6930,136,-9.53265,262
4,2002,Alabama St.,68.1969,223,63.0507,305,94.9003,264,90.1231,312,93.9840,26,102.7400,159,-12.61680,272


In [74]:
kenpom_pd = pd.read_csv(kenpom_dir + "/Misc_stats_def/misc02.csv")
kenpom_pd.head()

,Season,TeamName,FG2Pct,RankFG2Pct,FG3Pct,RankFG3Pct,FTPct,RankFTPct,BlockPct,RankBlockPct,...,RankOppF3GRate,ARate,RankARate,OppARate,RankOppARate,StlRate,RankStlRate,OppStlRate,RankOppStlRate,DefensiveFingerprint
0,2002,Air Force,54.8061,4,37.2760,58,71.1828,92,6.8287,235,...,64,61.3508,48,47.3958,11,0.1412,11,0.1084,218,18.6
1,2002,Akron,48.4271,136,37.0600,62,75.7576,12,6.9726,226,...,201,57.3508,115,55.3594,167,0.0868,263,0.1140,260,2.0
2,2002,Alabama,50.6579,56,30.8677,290,73.5782,40,8.0468,176,...,191,50.5495,267,46.9880,8,0.1053,130,0.0881,53,7.9
3,2002,Alabama A&M,44.2636,284,31.5871,272,66.4157,253,9.4669,107,...,176,52.7370,217,52.4425,92,0.1665,1,0.1039,175,10.2
4,2002,Alabama St.,45.8801,244,33.4646,209,64.4444,286,5.7722,280,...,9,44.2424,315,48.7465,32,0.0985,181,0.1358,321,13.4


# Generating Necessary Dataframes

In [75]:
reg_season_compact_pd = pd.read_csv(os.path.join(kaggle_dir,'RegularSeasonCompactResults.csv'))
reg_season_compact_pd.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [76]:
conference_tourney_compact_pd = pd.read_csv(os.path.join(kaggle_dir,'ConferenceTourneyGames.csv'))
conference_tourney_compact_pd.head()

,Season,ConfAbbrev,DayNum,WTeamID,LTeamID
0,2001,a_sun,121,1194,1144
1,2001,a_sun,121,1416,1240
2,2001,a_sun,122,1209,1194
3,2001,a_sun,122,1359,1239
4,2001,a_sun,122,1391,1273


In [77]:
NCAA_tourney_compact_pd = pd.read_csv(os.path.join(kaggle_dir,'NCAATourneyCompactResults.csv'))
NCAA_tourney_compact_pd.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [78]:
secondary_tourney_compact_pd = pd.read_csv(os.path.join(kaggle_dir,'NCAATourneyCompactResults.csv'))
secondary_tourney_compact_pd.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [79]:
teams_pd = pd.read_csv(os.path.join(kaggle_dir,'teams.csv'))
teamList = teams_pd['TeamName'].tolist()
teams_pd.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018


In [80]:
conferences_pd = pd.read_csv(os.path.join(kaggle_dir,'conferences.csv'))
conferences_pd.head()

,ConfAbbrev,Description
0,a_sun,Atlantic Sun Conference
1,a_ten,Atlantic 10 Conference
2,aac,American Athletic Conference
3,acc,Atlantic Coast Conference
4,aec,America East Conference


In [81]:
TeamConferences_pd = pd.read_csv(os.path.join(kaggle_dir,'TeamConferences.csv'))
TeamConferences_pd.head()

,Season,TeamID,ConfAbbrev
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


# Functions below are for getting/checking specific stats

In [82]:
def getTeamName(team_id):
    return teams_pd.loc[teams_pd["TeamID"] == team_id].values[0][1]
getTeamName(1105)

'Alabama A&M'

In [83]:
def getTeamID(team_name):
    return teams_pd.loc[teams_pd['TeamName'] == team_name].values[0][0]
getTeamID('Air Force')

1102

## Functions for handling the differences in naming btw the csvs

In [84]:
def handleKemponCsvs(df):
    #take out East Tennessee St.
    
    #some in kempon have quotes some dont, but shoulnt be a problem
    #For 2002
    #no Abilene Chr
    #no Alliant Intl
    #no Augusta
    #no Brooklyn
    #no Bryant
    #no Cent Arkansas
    #no Citadel
    #no Edwardsville
    #no ETSU
    #no 1167,CS Bakersfield,2008,2018
    #no FL Gulf Coast
    # 1168,CS Fullerton,1985,2018
    # 1169,CS Northridge,1991,2018
    # 1170,CS Sacramento,1992,2018
    #no tam c christi
    #ok im gonna stop tracking stuff thats missing
    df['TeamName'] = df['TeamName'].replace("St.", "St", regex=True)
    df['TeamName'] = df['TeamName'].replace("Albany", "Albany NY")
    df['TeamName'] = df['TeamName'].replace("American", "American Univ")
    df['TeamName'] = df['TeamName'].replace("Arkansas Little Rock", "Ark Little Rock")
    df['TeamName'] = df['TeamName'].replace("Arkansas Pine Bluff", "Ark Pine Bluff")
    df['TeamName'] = df['TeamName'].replace("Bethune Cookman", "Bethune-Cookman")
    df['TeamName'] = df['TeamName'].replace("Birmingham Southern", "Birmingham So")
    df['TeamName'] = df['TeamName'].replace("Boston University", "Boston Univ")
    df['TeamName'] = df['TeamName'].replace("Cal Poly", "Cal Poly SLO")
    df['TeamName'] = df['TeamName'].replace("Central Michigan", "C Michigan")
    df['TeamName'] = df['TeamName'].replace("Central Connecticut", "Central Conn")
    df['TeamName'] = df['TeamName'].replace("Charleston Southern", "Charleston So")
    df['TeamName'] = df['TeamName'].replace("Coastal Carolina", "Coastal Car")
    df['TeamName'] = df['TeamName'].replace("College of Charleston", "Col Charleston")
    df['TeamName'] = df['TeamName'].replace("Eastern Illinois", "E Illinois")
    df['TeamName'] = df['TeamName'].replace("Eastern Kentucky", "E Kentucky")
    df['TeamName'] = df['TeamName'].replace("Eastern Michigan", "E Michigan")
    df['TeamName'] = df['TeamName'].replace("Eastern Washington", "E Washington")
    df['TeamName'] = df['TeamName'].replace("Fairleigh Dickinson", "F Dickinson")
    df['TeamName'] = df['TeamName'].replace("Florida Atlantic", "FL Atlantic")
    df['TeamName'] = df['TeamName'].replace("George Washington", "G Washington")
    df['TeamName'] = df['TeamName'].replace("Georgia Southern", "Ga Southern")
    df['TeamName'] = df['TeamName'].replace("Illinois Chicago", "IL Chicago")
    df['TeamName'] = df['TeamName'].replace("Kent St", "Kent")
    df['TeamName'] = df['TeamName'].replace("Loyola Marymount", "Loy Marymount")
    df['TeamName'] = df['TeamName'].replace("Loyola Chicago", "Loyola-Chicago")
    df['TeamName'] = df['TeamName'].replace("Maryland Eastern Shore", "MD E Shore")
    df['TeamName'] = df['TeamName'].replace("Mississippi Valley St", "MS Valley St")
    df['TeamName'] = df['TeamName'].replace("Northern Illinois", "N Illinois")
    df['TeamName'] = df['TeamName'].replace("North Carolina A&T", "NC A&T")
    df['TeamName'] = df['TeamName'].replace("North Carolina St", "NC State")
    df['TeamName'] = df['TeamName'].replace("Prairie View A&M", "Prairie View")
    df['TeamName'] = df['TeamName'].replace("South Carolina St", "S Carolina St")
    df['TeamName'] = df['TeamName'].replace("Southeastern Louisiana", "SE Louisiana")
    df['TeamName'] = df['TeamName'].replace("Southeast Missouri St", "SE Missouri St")
    df['TeamName'] = df['TeamName'].replace("Stephen F. Austin", "SF Austin")
    df['TeamName'] = df['TeamName'].replace("Tennessee Martin", "TN Martin")
    df['TeamName'] = df['TeamName'].replace("Troy St", "Troy")
    df['TeamName'] = df['TeamName'].replace("Texas Southern", "TX Southern")
    df['TeamName'] = df['TeamName'].replace("Western Carolina", "W Carolina")
    df['TeamName'] = df['TeamName'].replace("Western Illinois", "W Illinois")
    df['TeamName'] = df['TeamName'].replace("Western Kentucky", "WKU")
    df['TeamName'] = df['TeamName'].replace("Western Michigan", "W Michigan")
    #holy fuck dammit my eyes hurt
    return df

In [85]:
def handleSrCsvs(df):
    # The stats CSV is a lit different in terms of naming so below is just some data cleaning
    df['School'] = df['School'].replace('(State)', 'St', regex=True)
    df['School'] = df['School'].replace('St.', 'St', regex=True) 
    df['School'] = df['School'].replace('Albany (NY)', 'Albany NY') 
    df['School'] = df['School'].replace('Boston University', 'Boston Univ')
    df['School'] = df['School'].replace('Central Michigan', 'C Michigan')
    df['School'] = df['School'].replace('(Eastern)', 'E', regex=True)
    df['School'] = df['School'].replace('Louisiana St', 'LSU')
    df['School'] = df['School'].replace('North Carolina St', 'NC State')
    df['School'] = df['School'].replace('Southern California', 'USC')
    df['School'] = df['School'].replace('University of California', 'California', regex=True) 
    df['School'] = df['School'].replace('American', 'American Univ')
    df['School'] = df['School'].replace('Arkansas-Little Rock', 'Ark Little Rock')
    df['School'] = df['School'].replace('Arkansas-Pine Bluff', 'Ark Pine Bluff')
    df['School'] = df['School'].replace('Bowling Green St', 'Bowling Green')
    df['School'] = df['School'].replace('Brigham Young', 'BYU')
    df['School'] = df['School'].replace('Cal Poly', 'Cal Poly SLO')
    df['School'] = df['School'].replace('Centenary (LA)', 'Centenary')
    df['School'] = df['School'].replace('Central Connecticut St', 'Central Conn')
    df['School'] = df['School'].replace('Charleston Southern', 'Charleston So')
    df['School'] = df['School'].replace('Coastal Carolina', 'Coastal Car')
    df['School'] = df['School'].replace('College of Charleston', 'Col Charleston')
    df['School'] = df['School'].replace('Cal St Fullerton', 'CS Fullerton')
    df['School'] = df['School'].replace('Cal St Sacramento', 'CS Sacramento')
    df['School'] = df['School'].replace('Cal St Bakersfield', 'CS Bakersfield')
    df['School'] = df['School'].replace('Cal St Northridge', 'CS Northridge')
    df['School'] = df['School'].replace('East Tennessee St', 'ETSU')
    df['School'] = df['School'].replace('Detroit Mercy', 'Detroit')
    df['School'] = df['School'].replace('Fairleigh Dickinson', 'F Dickinson')
    df['School'] = df['School'].replace('Florida Atlantic', 'FL Atlantic')
    df['School'] = df['School'].replace('Florida Gulf Coast', 'FL Gulf Coast')
    df['School'] = df['School'].replace('Florida International', 'Florida Intl')
    df['School'] = df['School'].replace('George Washington', 'G Washington')
    df['School'] = df['School'].replace('Georgia Southern', 'Ga Southern')
    df['School'] = df['School'].replace('Gardner-Webb', 'Gardner Webb')
    df['School'] = df['School'].replace('Illinois-Chicago', 'IL Chicago')
    df['School'] = df['School'].replace('Kent St', 'Kent')
    df['School'] = df['School'].replace('Long Island University', 'Long Island')
    df['School'] = df['School'].replace('Loyola Marymount', 'Loy Marymount')
    df['School'] = df['School'].replace('Loyola (MD)', 'Loyola MD')
    df['School'] = df['School'].replace('Loyola (IL)', 'Loyola-Chicago')
    df['School'] = df['School'].replace('Massachusetts', 'MA Lowell')
    df['School'] = df['School'].replace('Maryland-Eastern Shore', 'MD E Shore')
    df['School'] = df['School'].replace('Miami (FL)', 'Miami FL')
    df['School'] = df['School'].replace('Miami (OH)', 'Miami OH')
    df['School'] = df['School'].replace('Missouri-Kansas City', 'Missouri KC')
    df['School'] = df['School'].replace('Monmouth', 'Monmouth NJ')
    df['School'] = df['School'].replace('Mississippi Valley St', 'MS Valley St')
    df['School'] = df['School'].replace('Montana St', 'MTSU')
    df['School'] = df['School'].replace('Northern Colorado', 'N Colorado')
    df['School'] = df['School'].replace('North Dakota St', 'N Dakota St')
    df['School'] = df['School'].replace('Northern Illinois', 'N Illinois')
    df['School'] = df['School'].replace('Northern Kentucky', 'N Kentucky')
    df['School'] = df['School'].replace('North Carolina A&T', 'NC A&T')
    df['School'] = df['School'].replace('North Carolina Central', 'NC Central')
    df['School'] = df['School'].replace('Pennsylvania', 'Penn')
    df['School'] = df['School'].replace('South Carolina St', 'S Carolina St')
    df['School'] = df['School'].replace('Southern Illinois', 'S Illinois')
    df['School'] = df['School'].replace('UC-Santa Barbara', 'Santa Barbara')
    df['School'] = df['School'].replace('Southeastern Louisiana', 'SE Louisiana')
    df['School'] = df['School'].replace('Southeast Missouri St', 'SE Missouri St')
    df['School'] = df['School'].replace('Stephen F. Austin', 'SF Austin')
    df['School'] = df['School'].replace('Southern Methodist', 'SMU')
    df['School'] = df['School'].replace('Southern Mississippi', 'Southern Miss')
    df['School'] = df['School'].replace('Southern', 'Southern Univ')
    df['School'] = df['School'].replace('St. Bonaventure', 'St Bonaventure')
    df['School'] = df['School'].replace('St. Francis (NY)', 'St Francis NY')
    df['School'] = df['School'].replace('Saint Francis (PA)', 'St Francis PA')
    df['School'] = df['School'].replace('St. John\'s (NY)', 'St John\'s')
    df['School'] = df['School'].replace('Saint Joseph\'s', 'St Joseph\'s PA')
    df['School'] = df['School'].replace('Saint Louis', 'St Louis')
    df['School'] = df['School'].replace('Saint Mary\'s (CA)', 'St Mary\'s CA')
    df['School'] = df['School'].replace('Mount Saint Mary\'s', 'Mt St Mary\'s')
    df['School'] = df['School'].replace('Saint Peter\'s', 'St Peter\'s')
    df['School'] = df['School'].replace('Texas A&M-Corpus Christian', 'TAM C. Christian')
    df['School'] = df['School'].replace('Texas Christian', 'TCU')
    df['School'] = df['School'].replace('Tennessee-Martin', 'TN Martin')
    df['School'] = df['School'].replace('Texas-Rio Grande Valley', 'UTRGV')
    df['School'] = df['School'].replace('Texas Southern', 'TX Southern')
    df['School'] = df['School'].replace('Alabama-Birmingham', 'UAB')
    df['School'] = df['School'].replace('UC-Davis', 'UC Davis')
    df['School'] = df['School'].replace('UC-Irvine', 'UC Irvine')
    df['School'] = df['School'].replace('UC-Riverside', 'UC Riverside')
    df['School'] = df['School'].replace('Central Florida', 'UCF')
    df['School'] = df['School'].replace('Louisiana-Lafayette', 'ULL')
    df['School'] = df['School'].replace('Louisiana-Monroe', 'ULM')
    df['School'] = df['School'].replace('Maryland-Baltimore County', 'UMBC')
    df['School'] = df['School'].replace('North Carolina-Asheville', 'UNC Asheville')
    df['School'] = df['School'].replace('North Carolina-Greensboro', 'UNC Greensboro')
    df['School'] = df['School'].replace('North Carolina-Wilmington', 'UNC Wilmington')
    df['School'] = df['School'].replace('Nevada-Las Vegas', 'UNLV')
    df['School'] = df['School'].replace('Texas-Arlington', 'UT Arlington')
    df['School'] = df['School'].replace('Texas-San Antonio', 'UT San Antonio')
    df['School'] = df['School'].replace('Texas-El Paso', 'UTEP')
    df['School'] = df['School'].replace('Virginia Commonwealth', 'VA Commonwealth')
    df['School'] = df['School'].replace('Western Carolina', 'W Carolina')
    df['School'] = df['School'].replace('Western Illinois', 'W Illinois')
    df['School'] = df['School'].replace('Western Kentucky', 'WKU')
    df['School'] = df['School'].replace('Western Michigan', 'W Michigan')
    df['School'] = df['School'].replace('Abilene Christian', 'Abilene Chr')
    df['School'] = df['School'].replace('Montana State', 'Montana St')
    df['School'] = df['School'].replace('Central Arkansas', 'Cent Arkansas')
    df['School'] = df['School'].replace('Houston Baptist', 'Houston Bap')
    df['School'] = df['School'].replace('South Dakota St', 'S Dakota St')
    df['School'] = df['School'].replace('Maryland-Eastern Shore', 'MD E Shore')
    return df

In [86]:
#list of power six conferences
PowerSix = ['sec', 'acc', 'big_east', 'pac_12', 'pac_10', 'big_10', 'big_12']

In [87]:
def checkPower6(team_name, year):
    team_id = getTeamID(team_name)
    year_conference_pd = TeamConferences_pd[TeamConferences_pd['Season'] == int(year)]
    conference = year_conference_pd[year_conference_pd['TeamID'] == team_id].values[0][2]
    if conference in PowerSix:
        return 1
    else:
        return 0

In [88]:
def getNumWins(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    sr_pd = pd.read_csv(os.path.join(sr_dir, season + "_basic.csv"), skiprows=1)
    sr_pd = handleSrCsvs(sr_pd)
    if len(sr_pd.loc[sr_pd["School"] == team_name].index) == 0:
        return "missing"
    return sr_pd.loc[sr_pd["School"] == team_name].values[0][3]
getNumWins("Air Force", "2002")

9

In [89]:
def getWinPercentage(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    sr_pd = pd.read_csv(os.path.join(sr_dir, season + "_basic.csv"), skiprows=1)
    sr_pd = handleSrCsvs(sr_pd)
    if len(sr_pd.loc[sr_pd["School"] == team_name].index) == 0:
        return "missing"
    return sr_pd.loc[sr_pd["School"] == team_name].values[0][5]
getWinPercentage("Air Force", "2002")

0.321

In [90]:
def getPpg(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    sr_pd = pd.read_csv(os.path.join(sr_dir, season + "_basic.csv"), skiprows=1)
    sr_pd = handleSrCsvs(sr_pd)
    if len(sr_pd.loc[sr_pd["School"] == team_name].index) == 0:
        return "missing"
    return sr_pd.loc[sr_pd["School"] == team_name].values[0][14] / sr_pd.loc[sr_pd["School"] == team_name].values[0][2]
getPpg("Air Force", "2002")

57.32142857142857

In [91]:
def getAdjTempo(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    kenpom_pd = pd.read_csv(os.path.join(kenpom_dir, "summaries", "summary" + season[2:] + ".csv"))
    kenpom_pd = handleKemponCsvs(kenpom_pd)
    if len(kenpom_pd.loc[kenpom_pd["TeamName"] == team_name].index) == 0:
        return "missing"
    return kenpom_pd.loc[kenpom_pd["TeamName"] == team_name].values[0][4]
getAdjTempo("Air Force", "2002")

54.1733

In [92]:
def getTurnoverRate(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    sr_pd = pd.read_csv(os.path.join(sr_dir, season + "_adv.csv"), skiprows=1)
    sr_pd = handleSrCsvs(sr_pd)
    if len(sr_pd.loc[sr_pd["School"] == team_name].index) == 0:
        return "missing"
    return sr_pd.loc[sr_pd["School"] == team_name].values[0][27] / 100
getTurnoverRate("Air Force", "2002")

0.221

In [93]:
def getEfg(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    sr_pd = pd.read_csv(os.path.join(sr_dir, season + "_adv.csv"), skiprows=1)
    sr_pd = handleSrCsvs(sr_pd)
    if len(sr_pd.loc[sr_pd["School"] == team_name].index) == 0:
        return "missing"
    return sr_pd.loc[sr_pd["School"] == team_name].values[0][26]
getEfg("Air Force",  "2002")

0.5529999999999999

In [94]:
def getFtm(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    sr_pd = pd.read_csv(os.path.join(sr_dir, season + "_basic.csv"), skiprows=1)
    sr_pd = handleSrCsvs(sr_pd)
    if len(sr_pd.loc[sr_pd["School"] == team_name].index) == 0:
        return "missing"
    return sr_pd.loc[sr_pd["School"] == team_name].values[0][24]
getFtm("Air Force", "2002")

331

In [95]:
def getFtPercent(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    sr_pd = pd.read_csv(os.path.join(sr_dir, season + "_basic.csv"), skiprows=1)
    sr_pd = handleSrCsvs(sr_pd)
    if len(sr_pd.loc[sr_pd["School"] == team_name].index) == 0:
        return "missing"
    return sr_pd.loc[sr_pd["School"] == team_name].values[0][26]
getFtPercent("Air Force", "2002")

0.7120000000000001

In [96]:
def getStlPercentDef(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    kenpom_pd = pd.read_csv(os.path.join(kenpom_dir, "Misc_stats_def", "misc" + season[2:] + ".csv"))
    kenpom_pd = handleKemponCsvs(kenpom_pd)
    if len(kenpom_pd.loc[kenpom_pd["TeamName"] == team_name].index) == 0:
        return "missing"
    return kenpom_pd.loc[kenpom_pd["TeamName"] == team_name].values[0][26]
getStlPercentDef("Marshall", "2004")

0.0824

In [97]:
def getBlkPercentDef(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    kenpom_pd = pd.read_csv(os.path.join(kenpom_dir, "Misc_stats_def", "misc" + season[2:] + ".csv"))
    kenpom_pd = handleKemponCsvs(kenpom_pd)
    if len(kenpom_pd.loc[kenpom_pd["TeamName"] == team_name].index) == 0:
        return "missing"
    return kenpom_pd.loc[kenpom_pd["TeamName"] == team_name].values[0][8] / 100
getBlkPercentDef("Pittsburgh", "2002") 

0.119175

In [98]:
def getFgPercent(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    sr_pd = pd.read_csv(os.path.join(sr_dir, season + "_basic.csv"), skiprows=1)
    sr_pd = handleSrCsvs(sr_pd)
    if len(sr_pd.loc[sr_pd["School"] == team_name].index) == 0:
        return "missing"
    return sr_pd.loc[sr_pd["School"] == team_name].values[0][20]
getFgPercent("Air Force", "2002")

0.46299999999999997

In [99]:
def get3pp(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    sr_pd = pd.read_csv(os.path.join(sr_dir, season + "_basic.csv"), skiprows=1)
    sr_pd = handleSrCsvs(sr_pd)
    if len(sr_pd.loc[sr_pd["School"] == team_name].index) == 0:
        return "missing"
    return sr_pd.loc[sr_pd["School"] == team_name].values[0][23]
get3pp("Air Force", "2002")

0.373

In [100]:
def get3pm(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
    sr_pd = pd.read_csv(os.path.join(sr_dir, season + "_basic.csv"), skiprows=1)
    sr_pd = handleSrCsvs(sr_pd)
    if len(sr_pd.loc[sr_pd["School"] == team_name].index) == 0:
        return "missing"
    return sr_pd.loc[sr_pd["School"] == team_name].values[0][21]
get3pm("Air Force", "2002")

208

### Create a dictionary of all the stat getters

In [101]:
stat_getters = {}
stat_getters[0] = checkPower6
stat_getters[1] = getNumWins
stat_getters[2] = getWinPercentage
stat_getters[3] = getPpg
stat_getters[4] = getAdjTempo
stat_getters[5] = getTurnoverRate
stat_getters[6] = getEfg
stat_getters[7] = getFtm
stat_getters[8] = getFtPercent
stat_getters[9] = getStlPercentDef
stat_getters[10] = getBlkPercentDef
stat_getters[11] = getFgPercent
stat_getters[12] = get3pp
stat_getters[13] = get3pm

## Additional Game data getters

In [102]:
def getLoc(row):
    if (row == 'H'):
        home = 1
    if (row == 'A'):
        home = -1
    if (row == 'N'):
        home = 0
    return home

# Getting the Season Vectors for each team and year :0

In [103]:
#creates the stat vector for a given team during a give season/year
def getSeasonVector(team_name, season):
    assert len(season) == 4, "Season format must be the ending year, and a string; eg: 2002"
    
#     # The data frame below holds stats for every single game in the given season
#     season_data = reg_season_compact_pd[reg_season_compact_pd['Season'] == year]
#     print(season_data.head())
#     #number of games in the season
#     num_games = len(season_data.index)
    
    team_vector = []
    for i in range(num_team_stats):
        team_vector.append(stat_getters.get(i)(team_name, season))
    return np.array(team_vector)
getSeasonVector("Air Force", "2002")

array([0.00000000e+00, 9.00000000e+00, 3.21000000e-01, 5.73214286e+01,
       5.41733000e+01, 2.21000000e-01, 5.53000000e-01, 3.31000000e+02,
       7.12000000e-01, 1.41200000e-01, 6.82870000e-02, 4.63000000e-01,
       3.73000000e-01, 2.08000000e+02])

# Make a dictionary of every team and the respective season vector    

In [104]:
def createSeasonDict(season):
    seasonDict = collections.defaultdict(list)
    for team in teams_list:
        team_vector = getSeasonVector(team, season)
        
        #if any of the entries in the vector equals 0, then the team name wasn't present in one of the data files
        if not any(map(lambda x: x == "missing", team_vector)):
            seasonDict[team] = team_vector
        else:
            print(team)
            print(team_vector)
    return seasonDict

In [105]:
def createTrainingSet(years):
    #calculating the dimensions of the training sets
    totalNumGames = 0
    for year in years:
        reg_season = reg_season_compact_pd[reg_season_compact_pd['Season'] == year]
        NCAA_tourney = NCAA_tourney_compact_pd[NCAA_tourney_compact_pd['Season'] == year]
        secondary_tourney = secondary_tourney_compact_pd[secondary_tourney_compact_pd['Season'] == year]
        conference_tourney = conference_tourney_compact_pd[conference_tourney_compact_pd['Season'] == year]
        totalNumGames += (len(reg_season.index) + len(conference_tourney.index) + len(NCAA_tourney.index) + len(secondary_tourney.index))
        
    StatsLength = len(getSeasonData(1101, 2012))
    xTrain = np.zeros((totalNumGames, statsLength + 1))
    yTrain = np.zeros((totalNumGames))
    seasonCounter = 0
    
    #create a training set for each season to put into the x and y Train
    for year in years:
        #get the team vectors for the year in iteration
        team_vectors = createSeasonDict(year)
        
        #get all of the games for the year in iteration
        reg_season = reg_season_compact_pd[reg_season_compact_pd['Season'] == year]
        NCAA_tourney = NCAA_tourney_compact_pd[NCAA_tourney_compact_pd['Season'] == year]
        secondary_tourney = secondary_tourney_compact_pd[secondary_tourney_compact_pd['Season'] == year]
        conference_tourney = conference_tourney_compact_pd[conference_tourney_compact_pd['Season'] == year]
        
        games_in_season = len(reg_season.index) + len(NCAA_tourney.index) + len(secondary_tourney.index) + len(conference_tourney.index)
        xTrainSeason = np.zeros((games_in_season, StatsLength + 1))
        yTrainSeason = np.zeros((games_in_season))
        counter = 0
        
        #add each game to the training vectors from the regular season
        for index, row in reg_season.iterrows():
            #determine the difference vector for each game
            w_team = row['WteamID']
            w_vector = team_vectors[w_team]
            l_team = row['LteamID']
            l_vector = team_vectors[l_team]
            diff_vector = [a - b for a, b in zip(w_vector, l_vector)]
            home = getLoc(row['Wloc'])
            diff_vector.append(home)
            #flip every odd numbered game
            if (counter % 2 == 0):
                xTrainSeason[counter] = diff_vector
                yTrainSeason[counter] = 1
            else:
                xTrainSeason[counter] = [-x for x in diff_vector]
                yTrainSeason[counter] = 0
            counter += 1
            
        #add each game to the training vector from the NCAA tourney    
        for index, row in NCAA_tourney.iterrrows():
            #determine the diff vector for each game
            w_team = row['WteamID']
            w_vector = team_vectors[w_team]
            l_team = row['LteamID']
            l_vector = team_vectors[l_team]
            diff_vector = [a - b for a, b in zip(w_vector, l_vector)]
            home = 0
            #flip every odd numbered game
            if (counter % 2 == 0):
                xTrainSeason[counter] = diff_vector
                yTrainSeason[counter] = 1
            else:
                xTrainSeason[counter] = [-x for x in diff_vector]
                yTrainSeason[counter] = 0
            counter += 1
        
        #add each game to the training vector from the conference tourney
        for index, row in conference_tourney.iterrows():
            w_team = row['WteamID']
            w_vector = team_vectors[w_team]
            l_team = row['LteamID']
            l_vector = team_vectors[l_team]
            diff_vector = [a - b for a, b in zip(w_vector, l_vector)]
            home = 0
            #flip every odd numbered game
            if (counter % 2 == 0):
                xTrainSeason[counter] = diff_vector
                yTrainSeason[counter] = 1
            else:
                xTrainSeason[counter] = [-x for x in diff_vector]
                yTrainSeason[counter] = 0
            counter += 1
            
        #add each game to the training vector from the secondary tourneys
        for index, row in secondary_tourney.iterrows():
            #determine the difference vector for each game
            w_team = row['WteamID']
            w_vector = team_vectors[w_team]
            l_team = row['LteamID']
            l_vector = team_vectors[l_team]
            diff_vector = [a - b for a, b in zip(w_vector, l_vector)]
            home = getLoc(row['Wloc'])
            diff_vector.append(home)
            #flip every odd numbered game
            if (counter % 2 == 0):
                xTrainSeason[counter] = diff_vector
                yTrainSeason[counter] = 1
            else:
                xTrainSeason[counter] = [-x for x in diff_vector]
                yTrainSeason[counter] = 0
            counter += 1
            
        xTrain[indexCounter:numGamesInSeason+indexCounter] = xTrainSeason
        yTrain[indexCounter:numGamesInSeason+indexCounter] = yTrainSeason
        indexCounter += numGamesInSeason
    return xTrain, yTrain

In [419]:
createSeasonDict("2002")

Abilene Chr
['0' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing'
 'missing' 'missing' 'missing' 'missing' 'missing' 'missing']
Alabama
['1' 'missing' 'missing' 'missing' '68.0142' 'missing' 'missing' 'missing'
 'missing' '0.1053' '0.080468' 'missing' 'missing' 'missing']
Alcorn St
['0' 'missing' 'missing' 'missing' '72.9539' 'missing' 'missing' 'missing'
 'missing' '0.10800000000000001' '0.09434' 'missing' 'missing' 'missing']
Alliant Intl
['0' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing'
 'missing' 'missing' 'missing' 'missing' 'missing' 'missing']
Arizona
['1' 'missing' 'missing' 'missing' '71.1331' 'missing' 'missing' 'missing'
 'missing' '0.0835' '0.072512' 'missing' 'missing' 'missing']
Ark Little Rock
['0' 'missing' 'missing' 'missing' '69.1203' 'missing' 'missing' 'missing'
 'missing' '0.1304' '0.060389' 'missing' 'missing' 'missing']
Armstrong St
['0' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing'
 'missing' 

MA Lowell
['0' '13' '0.44799999999999995' '64.3103448275862' 'missing' '0.188'
 '0.48' '307' '0.669' 'missing' 'missing' '0.41700000000000004' '0.348'
 '204']
Marquette
['1' 'missing' 'missing' 'missing' '64.6162' 'missing' 'missing' 'missing'
 'missing' '0.1259' '0.100983' 'missing' 'missing' 'missing']
Maryland
['1' 'missing' 'missing' 'missing' '73.6587' 'missing' 'missing' 'missing'
 'missing' '0.1118' '0.136106' 'missing' 'missing' 'missing']
Massachusetts
['0' 'missing' 'missing' 'missing' '64.266' 'missing' 'missing' 'missing'
 'missing' '0.0889' '0.103182' 'missing' 'missing' 'missing']
McNeese St
['0' 'missing' 'missing' 'missing' '66.2135' 'missing' 'missing' 'missing'
 'missing' '0.1218' '0.133097' 'missing' 'missing' 'missing']
MD E Shore
['0' 'missing' 'missing' 'missing' '64.3362' 'missing' 'missing' 'missing'
 'missing' '0.0895' '0.055889999999999995' 'missing' 'missing' 'missing']
Miami FL
['0' 'missing' 'missing' 'missing' '69.0973' 'missing' 'missing' 'missing'
 'miss

St Mary's CA
['0' '9' '0.31' '61.03448275862069' 'missing' '0.207'
 '0.43799999999999994' '417' '0.677' 'missing' 'missing'
 '0.39899999999999997' '0.275' '121']
St Peter's
['0' '4' '0.14300000000000002' '65.14285714285714' 'missing' '0.22'
 '0.457' '352' '0.623' 'missing' 'missing' '0.419' '0.31' '120']
Stanford
['1' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing'
 'missing' 'missing' 'missing' 'missing' 'missing' 'missing']
Stetson
['0' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing'
 'missing' 'missing' 'missing' 'missing' 'missing' 'missing']
Stony Brook
['0' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing'
 'missing' 'missing' 'missing' 'missing' 'missing' 'missing']
TAM C. Christi
['0' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing' 'missing'
 'missing' 'missing' 'missing' 'missing' 'missing' 'missing']
Texas
['1' 'missing' 'missing' 'missing' '68.5302' 'missing' 'missing' 'missing'
 'missing' '0.0869' '

defaultdict(list,
            {'Air Force': array([0.00000000e+00, 9.00000000e+00, 3.21000000e-01, 5.73214286e+01,
                    5.41733000e+01, 2.21000000e-01, 5.53000000e-01, 3.31000000e+02,
                    7.12000000e-01, 1.41200000e-01, 6.82870000e-02, 4.63000000e-01,
                    3.73000000e-01, 2.08000000e+02]),
             'Akron': array([0.0000000e+00, 1.0000000e+01, 3.2300000e-01, 6.9483871e+01,
                    6.5639400e+01, 1.8900000e-01, 5.0600000e-01, 5.5400000e+02,
                    7.5800000e-01, 8.6800000e-02, 6.9726000e-02, 4.4800000e-01,
                    3.7100000e-01, 1.8200000e+02]),
             'Alabama A&M': array([0.00000000e+00, 1.90000000e+01, 6.55000000e-01, 7.99310345e+01,
                    7.89833000e+01, 1.77000000e-01, 4.58000000e-01, 4.78000000e+02,
                    6.72000000e-01, 1.66500000e-01, 9.46690000e-02, 4.05000000e-01,
                    3.11000000e-01, 2.12000000e+02]),
             'Alabama St': array([0.00000